In [15]:
import pandas as pd

df = pd.read_csv('playbook.csv', parse_dates=True, names=['contract', 'direction', 'vol'])
df

,contract,direction,vol
2021-08-30,510050C2109M03000,L,20
2021-08-30,510050C2109M03000,S,20
2021-08-30,510050C2110M02800,L,50
2021-08-30,510050C2110M03000,S,50
2021-08-30,M0C5,L,20
2021-08-30,M0C6,S,20
2021-08-30,M1C4,L,50
2021-08-30,M1C6,S,50


In [32]:
time = pd.Timestamp('20010101')
time + pd.Timedelta('1d')
for row in df[df.index == pd.Timestamp('20210830')].itertuples(index=False):
    print(row.contract, row.direction)

 510050C2109M03000  L
 510050C2109M03000  S
 510050C2110M02800  L
 510050C2110M03000  S
 M0C5  L
 M0C6  S
 M1C4  L
 M1C6  S


In [207]:
class Condition(dict):
    def __init__(self, type, **kargs) -> None:
        self.type = type
        super().__init__(kargs)

c = Condition('a', **{'x':1, 'y':2})
c

{'x': 1, 'y': 2}

In [226]:
dfa = pd.DataFrame(range(100, 110), pd.date_range('20000101', periods=10, freq='D'))
dfb = pd.DataFrame({'a': range(0,10), 'b': pd.date_range('20000101', periods=10)})
pd.merge(dfa, dfb, left_index=True, right_on='b')


,0,a,b
0,0,0,2000-01-01
1,1,1,2000-01-02
2,2,2,2000-01-03
3,3,3,2000-01-04
4,4,4,2000-01-05
5,5,5,2000-01-06
6,6,6,2000-01-07
7,7,7,2000-01-08
8,8,8,2000-01-09
9,9,9,2000-01-10


In [194]:
from collections import namedtuple
Trade = namedtuple('Trade', 'name symbol direction vol date trigger', defaults=['', '', 'L', 0, None, None])
d = Trade._field_defaults
d['a'] = 0
Trade(**d)

TypeError: __new__() got an unexpected keyword argument 'a'

In [214]:
import yaml
with open('playbook.yml') as f:
    res = yaml.safe_load(f)
    print(res['accounts'][1]['trades'][1]['closePolicy'])
    print(res['accounts'][1]['trades'][1])

[{'type': 'pl', 'expression': '$>0.8'}, {'type': 'linked', 'position': 'primary', 'on': 'close'}]
{'date': datetime.date(2022, 3, 25), 'symbol': '510050C2204M02900', 'direction': 'S', 'volume': 20, 'closePolicy': [{'type': 'pl', 'expression': '$>0.8'}, {'type': 'linked', 'position': 'primary', 'on': 'close'}]}


In [219]:
eval('$>1.0'.replace('$', str(1.234)))

True

In [78]:
import akshare as ak
df = ak.fund_etf_category_sina(symbol="ETF基金")

In [183]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

replay_df = pd.read_csv("res.csv", parse_dates=True)

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
    specs=[
        [{'secondary_y':True}], 
        [{}]])

spot_tr = go.Scatter(x=replay_df.index, y=replay_df['spot'], name='spot')
pl_amt_tr = go.Scatter(x=replay_df.index, y=replay_df['pl_amt'], name='p&l-amount')
fig.add_traces([spot_tr, pl_amt_tr], rows=1, cols=1, secondary_ys=[True, False])

pl_pct_tr = go.Scatter(x=replay_df.index, y=replay_df['pl_pct']*100, name='p&l-percent')
delta_tr = go.Scatter(x=replay_df.index, y=replay_df['delta'], name='delta')
long_vol_tr = go.Bar(x=replay_df.index, y=replay_df['long_vol'], name='long vol')
short_vol_tr = go.Bar(x=replay_df.index, y=replay_df['short_vol'], name='short vol')
fig.add_traces([pl_pct_tr, delta_tr, long_vol_tr, short_vol_tr], rows=2, cols=1)

fig['data'][2]['yaxis'] = 'y4'
fig['data'][3]['yaxis'] = 'y5'
fig['layout']['yaxis4']=dict(
        title="pl",
        overlaying="y3",
        anchor="x2",
        side="right"
        #position=0.15
    )
fig['layout']['yaxis5']=dict(
        title="delta",
        anchor="free",
        overlaying='y3',
        range=[-1, 1],
        side="right",
        position=1
    )
fig.update_layout(barmode='stack', height=600, title='replay')
fig.show()

In [184]:

print(fig['layout'])

for i in range(len(fig['data'])):
    print(i, fig['data'][i].name, fig['data'][i]['xaxis'], fig['data'][i]['yaxis'])

Layout({
    'barmode': 'stack',
    'height': 600,
    'template': '...',
    'title': {'text': 'replay'},
    'xaxis': {'anchor': 'y', 'domain': [0.0, 0.94], 'matches': 'x2', 'showticklabels': False},
    'xaxis2': {'anchor': 'y3', 'domain': [0.0, 0.94]},
    'yaxis': {'anchor': 'x', 'domain': [0.575, 1.0]},
    'yaxis2': {'anchor': 'x', 'overlaying': 'y', 'side': 'right'},
    'yaxis3': {'anchor': 'x2', 'domain': [0.0, 0.425]},
    'yaxis4': {'anchor': 'x2', 'overlaying': 'y3', 'side': 'right', 'title': {'text': 'pl'}},
    'yaxis5': {'anchor': 'free',
               'overlaying': 'y3',
               'position': 1,
               'range': [-1, 1],
               'side': 'right',
               'title': {'text': 'delta'}}
})
0 spot x y2
1 p&l-amount x y
2 p&l-percent x2 y4
3 delta x2 y5
4 long vol x2 y3
5 short vol x2 y3
